In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]       = Query[['a','z']]
    Src[['Symbol','TF']] = Query[['symbol','tf']]
    
    Src['Datetime']     = Query['datetime']
    Src['Date']         = Query['datetime'].dt.date
    Src['Time']         = Query['datetime'].dt.time
    
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc    = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Time+1']      = (Calc['Datetime'] + pd.Timedelta(hours=1)).dt.time

    Calc['Day Open']    = Calc_by['Open']  .transform('first')
    Calc['Day High']    = Calc_by['High']  .transform('max')
    Calc['Day Low']     = Calc_by['Low']   .transform('min')
    Calc['Day Close']   = Calc_by['Close'] .transform('last')
    Calc['Day Hilo']    = Calc['Day High'] - Calc['Day Low']

    Calc['ID High']     = Calc_by['High'] .expanding().max()    .reset_index(drop=1)
    Calc['ID Low']      = Calc_by['Low']  .expanding().min()    .reset_index(drop=1)
    Calc['ID Hilo']     = Calc['ID High'] - Calc['ID Low']

    Calc['Bwd HL']      = Calc['ID Hilo']

    Calc['Bwd Chg']     =                     Calc['Open'] - Calc['Day Open']
    Calc['Fwd Chg']     = Calc['Day Close'] - Calc['Open']

    Calc['Bwd Chg Abs'] = Calc['Bwd Chg'].abs()
    Calc['Fwd Chg Abs'] = Calc['Fwd Chg'].abs()


    Calc['Fwd Bull']    = Calc['Fwd Chg'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['Fwd Bear']    = Calc['Fwd Chg'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['Fwd Chg Pos'] = Calc['Fwd Chg Abs'] * Calc['Fwd Bull']
    Calc['Fwd Chg Neg'] = Calc['Fwd Chg Abs'] * Calc['Fwd Bear']

    Calc['Fwd HL']      = Calc['Day Hilo'] - Calc['ID Hilo']
    Calc['Fwd HL Pos']  = Calc['Fwd HL'] * Calc['Fwd Bull']
    Calc['Fwd HL Neg']  = Calc['Fwd HL'] * Calc['Fwd Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats 1

In [53]:
Calc.groupby('Time')[['Fwd Chg']].describe().round(0).astype(int)

Fwd Chg                                     
           count mean   std   min  25% 50%  75%   max
Time                                                 
09:00:00    1240  -22  1314 -5150 -851  50  826  4535
10:00:00    1240  -15  1214 -4330 -745  15  715  4395
11:00:00    1240  -27  1031 -4930 -565  25  578  3570
12:00:00    1240    4   889 -4640 -490  15  515  3600
13:00:00    1245   -7   799 -5280 -395  15  440  3760
14:00:00    1245   -9   711 -4730 -405   5  380  3520
15:00:00    1245   -9   657 -5395 -325   5  355  3920
16:00:00    1245   -1   525 -2520 -255  20  270  3815
17:00:00    1245   -6   360 -2840 -195  10  185  1370
18:00:00     682   13   220 -1705 -110  15  130   755

In [54]:
Calc.groupby('Time')[['Fwd Chg Abs']].describe().round(0).astype(int)

Fwd Chg Abs                                     
               count  mean  std min  25%  50%   75%   max
Time                                                     
09:00:00        1240  1031  814   0  420  845  1440  5150
10:00:00        1240   934  775   0  330  732  1361  4395
11:00:00        1240   775  680   0  269  570  1130  4930
12:00:00        1240   660  595   0  225  500   935  4640
13:00:00        1245   576  553   0  195  415   790  5280
14:00:00        1245   517  488   0  185  385   710  4730
15:00:00        1245   463  466   0  150  340   605  5395
16:00:00        1245   368  373   0  130  265   475  3815
17:00:00        1245   255  254   0   95  185   340  2840
18:00:00         682   158  153   0   55  125   214  1705

In [55]:
Calc.groupby('Time')[['Fwd Chg Pos']].describe().round(0).astype(int)

Fwd Chg Pos                                    
               count mean  std min  25%  50%   75%   max
Time                                                    
09:00:00         640  978  787   0  385  800  1396  4535
10:00:00         630  904  758   0  335  715  1289  4395
11:00:00         631  735  616   0  262  550  1045  3570
12:00:00         632  651  563   0  250  502   926  3600
13:00:00         639  554  514   0  190  420   755  3760
14:00:00         627  505  468   0  195  375   685  3520
15:00:00         627  451  434   0  155  350   590  3920
16:00:00         642  356  373   0  130  260   455  3815
17:00:00         637  244  224   0   95  180   330  1370
18:00:00         368  158  139   0   55  120   225   755

In [56]:
Calc.groupby('Time')[['Fwd Chg Neg']].describe().round(0).astype(int)

Fwd Chg Neg                                     
               count  mean  std   min   25%  50%  75% max
Time                                                     
09:00:00         600 -1088  839 -5150 -1490 -885 -479  -5
10:00:00         610  -964  792 -4330 -1418 -758 -330  -5
11:00:00         609  -816  738 -4930 -1200 -580 -275  -5
12:00:00         608  -669  628 -4640  -936 -500 -205  -5
13:00:00         606  -599  591 -5280  -824 -412 -200  -5
14:00:00         618  -530  508 -4730  -752 -405 -175  -5
15:00:00         618  -475  496 -5395  -624 -328 -145  -5
16:00:00         603  -381  374 -2520  -510 -265 -130  -5
17:00:00         608  -267  283 -2840  -355 -195  -94  -5
18:00:00         314  -157  168 -1705  -205 -125  -55  -5

In [ ]:
# Calc.groupby('Time')[['Fwd HL']].describe().round(0).astype(int)

In [ ]:
# Calc.groupby('Time')[['Fwd HL Pos']].describe().round(0).astype(int)

In [ ]:
# Calc.groupby('Time')[['Fwd HL Neg']].describe().round(0).astype(int)

# Stats 2

In [47]:
BINS_CHG = [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500]
BINS_HL  = [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000]

In [44]:
def STATS(Calc, HR, BINS, X, Y):
    pipe = []
    for b in BINS: 

        Df = Calc[(Calc['Time'] == dt.time(HR,00)) & (Calc[X] > b)]

        pipe.append({
            'Time =': dt.time(HR,00),
            f'{X} >': b,
            **np.round(Df[Y].describe(), 1) #.astype(int)
        })
    pass
    return pd.DataFrame(pipe)

In [48]:
STATS(Calc, HR=13, BINS=BINS_CHG, X='Bwd Chg Abs', Y='Fwd Chg Abs')

,Time =,Bwd Chg Abs >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,1148.0,579.5,561.0,0.0,195.0,415.0,786.2,5280.0
1,13:00:00,200,1037.0,578.1,546.2,0.0,190.0,415.0,790.0,3760.0
2,13:00:00,500,778.0,588.4,556.6,0.0,200.0,435.0,790.0,3760.0
3,13:00:00,800,527.0,611.5,558.8,0.0,200.0,470.0,815.0,3595.0
4,13:00:00,1000,406.0,639.2,569.5,5.0,220.0,495.0,853.8,3595.0
5,13:00:00,1200,280.0,640.0,566.4,5.0,220.0,482.5,891.2,3555.0
6,13:00:00,1500,147.0,717.1,633.5,5.0,242.5,570.0,1057.5,3555.0
7,13:00:00,1800,85.0,722.9,556.4,5.0,255.0,600.0,1075.0,2365.0
8,13:00:00,2000,54.0,657.8,518.0,5.0,228.8,567.5,925.0,2365.0
9,13:00:00,2200,29.0,773.6,578.5,5.0,285.0,770.0,1180.0,2365.0


In [49]:
STATS(Calc, HR=13, BINS=BINS_CHG, X='Bwd Chg Abs', Y='Fwd HL')

,Time =,Bwd Chg Abs >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,1148.0,335.0,481.5,0.0,0.0,150.0,491.2,4920.0
1,13:00:00,200,1037.0,330.1,467.9,0.0,0.0,145.0,480.0,4920.0
2,13:00:00,500,778.0,326.3,476.9,0.0,0.0,145.0,450.0,4920.0
3,13:00:00,800,527.0,345.2,465.0,0.0,0.0,160.0,500.0,2600.0
4,13:00:00,1000,406.0,345.9,458.9,0.0,0.0,167.5,500.0,2600.0
5,13:00:00,1200,280.0,341.6,467.0,0.0,0.0,147.5,491.2,2375.0
6,13:00:00,1500,147.0,367.8,495.2,0.0,0.0,165.0,520.0,2315.0
7,13:00:00,1800,85.0,386.8,509.4,0.0,0.0,125.0,635.0,2040.0
8,13:00:00,2000,54.0,330.9,461.7,0.0,0.0,35.0,598.8,1965.0
9,13:00:00,2200,29.0,358.4,498.4,0.0,0.0,35.0,635.0,1965.0


In [52]:
STATS(Calc, HR=13, BINS=BINS_HL, X='Bwd HL', Y='Fwd Chg Abs')

,Time =,Bwd HL >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,1245.0,576.1,553.2,0.0,195.0,415.0,790.0,5280.0
1,13:00:00,200,1245.0,576.1,553.2,0.0,195.0,415.0,790.0,5280.0
2,13:00:00,500,1245.0,576.1,553.2,0.0,195.0,415.0,790.0,5280.0
3,13:00:00,800,1201.0,582.1,557.4,0.0,195.0,420.0,790.0,5280.0
4,13:00:00,1000,1093.0,596.6,563.9,0.0,205.0,430.0,810.0,5280.0
5,13:00:00,1200,939.0,620.0,584.9,0.0,210.0,455.0,835.0,5280.0
6,13:00:00,1500,657.0,659.8,630.3,0.0,210.0,485.0,900.0,5280.0
7,13:00:00,1800,396.0,713.6,645.3,0.0,230.0,537.5,1051.2,3760.0
8,13:00:00,2000,280.0,732.4,647.2,0.0,243.8,557.5,1096.2,3760.0
9,13:00:00,2200,201.0,779.5,690.6,0.0,260.0,570.0,1165.0,3760.0


In [51]:
STATS(Calc, HR=13, BINS=BINS_HL, X='Bwd HL', Y='Fwd HL')

,Time =,Bwd HL >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,1245.0,337.0,485.2,0.0,0.0,150.0,500.0,4920.0
1,13:00:00,200,1245.0,337.0,485.2,0.0,0.0,150.0,500.0,4920.0
2,13:00:00,500,1245.0,337.0,485.2,0.0,0.0,150.0,500.0,4920.0
3,13:00:00,800,1201.0,337.3,489.6,0.0,0.0,145.0,500.0,4920.0
4,13:00:00,1000,1093.0,340.6,496.5,0.0,0.0,145.0,515.0,4920.0
5,13:00:00,1200,939.0,352.3,506.8,0.0,0.0,150.0,525.0,4920.0
6,13:00:00,1500,657.0,367.2,533.7,0.0,0.0,150.0,535.0,4920.0
7,13:00:00,1800,396.0,402.9,555.5,0.0,0.0,192.5,620.0,4920.0
8,13:00:00,2000,280.0,391.2,554.9,0.0,0.0,187.5,603.8,4920.0
9,13:00:00,2200,201.0,409.7,597.2,0.0,0.0,185.0,615.0,4920.0


# Snippets

In [ ]:
def CALCULATIONS(Src):
    Calc    = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Time+1']      = (Calc['Datetime'] + pd.Timedelta(hours=1)).dt.time

    Calc['Day Open']    = Calc_by['Open']  .transform('first')
    Calc['Day High']    = Calc_by['High']  .transform('max')
    Calc['Day Low']     = Calc_by['Low']   .transform('min')
    Calc['Day Close']   = Calc_by['Close'] .transform('last')
    Calc['Day Hilo']    = Calc['Day High'] - Calc['Day Low']

    Calc['ID High']     = Calc_by['High'] .expanding().max()    .reset_index(drop=1)
    Calc['ID Low']      = Calc_by['Low']  .expanding().min()    .reset_index(drop=1)
    Calc['ID Hilo']     = Calc['ID High'] - Calc['ID Low']

    Calc['Bwd HL']      = Calc['ID Hilo']

    Calc['Bwd Chg']     =                     Calc['Open'] - Calc['Day Open']
    Calc['Fwd Chg']     = Calc['Day Close'] - Calc['Open']

    Calc['Bwd Chg Abs'] = Calc['Bwd Chg'].abs()
    Calc['Fwd Chg Abs'] = Calc['Fwd Chg'].abs()


    Calc['Fwd Bull']    = Calc['Fwd Chg'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['Fwd Bear']    = Calc['Fwd Chg'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['Fwd Chg Pos'] = Calc['Fwd Chg Abs'] * Calc['Fwd Bull']
    Calc['Fwd Chg Neg'] = Calc['Fwd Chg Abs'] * Calc['Fwd Bear']

    Calc['Fwd HL']      = Calc['Day Hilo'] - Calc['ID Hilo']
    Calc['Fwd HL Pos']  = Calc['Fwd HL'] * Calc['Fwd Bull']
    Calc['Fwd HL Neg']  = Calc['Fwd HL'] * Calc['Fwd Bear']
    return Calc

In [ ]:
def STATS(Calc, HR, X, Y):
    pipe = []
    for b in [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000]:

        Df = Calc[(Calc['Time'] == dt.time(HR,00)) & (Calc[X] > b)]

        pipe.append({
            'Time =': dt.time(HR,00),
            f'{X} >': b,
            **Df[Y].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)


Calc.groupby('Time')[['Fwd Chg']].describe().round(0).astype(int)
Calc.groupby('Time')[['Fwd Chg Abs']].describe().round(0).astype(int)
Calc.groupby('Time')[['Fwd Chg Pos']].describe().round(0).astype(int)
Calc.groupby('Time')[['Fwd Chg Neg']].describe().round(0).astype(int)

Fwd Chg Neg                                     
               count  mean  std   min   25%  50%  75% max
Time                                                     
09:00:00         600 -1088  839 -5150 -1490 -885 -479  -5
10:00:00         610  -964  792 -4330 -1418 -758 -330  -5
11:00:00         609  -816  738 -4930 -1200 -580 -275  -5
12:00:00         608  -669  628 -4640  -936 -500 -205  -5
13:00:00         606  -599  591 -5280  -824 -412 -200  -5
14:00:00         618  -530  508 -4730  -752 -405 -175  -5
15:00:00         618  -475  496 -5395  -624 -328 -145  -5
16:00:00         603  -381  374 -2520  -510 -265 -130  -5
17:00:00         608  -267  283 -2840  -355 -195  -94  -5
18:00:00         314  -157  168 -1705  -205 -125  -55  -5